# Technical Analysis using TaLib

Roboquant comes out of box with support for technical analysis. It does this by having dedicate support for the popular technical analysis libraries TaLib. TaLib contains over 150 technical indicators. These are indicators ranging from RSI and Bollinger Bands to Candlestick pattern recognition. Directly using the TA-Lib library can be difficult due to its API, but roboquant makes it fairly easy by providing a convenient wrapper.

In [ ]:
%use roboquant(1.5.0-SNAPSHOT)
Welcome()

In [ ]:
// We first setup the feed we'll use in all examples below
val feed = AvroFeed.sp500()

We start by defining the  strategy using TALib indicators we want to test in combination with the `TaLibStrategy` class. There are three steps:

1) Create an instance of `TaLibStrategy` and define how much history it should maintain before invoking the indicators. If you choose this value too low, certain indicators will fail and an exception will be thrown. For example if you want a moving average over 20 days, the history should be at least 20. If you choose the history value too large, it will take longer before your strategy starts to generate signals since it will wait until it has enough history. For many of the technical indicators like candle stick patterns, 15 is sufficient (also the default value).

2) Define what you logic to use for generating BUY signals. This can be a combination of indicators and you can also include regular Kotlin code. The only restriction is that the last statement has to be a boolean; true if you want to BUY, false otherwise.

3) Define what you logic to use for generating SELL signals. The same rules and conditions apply as for the BUY method.

Please note, that although not common practice, the framework doesn't stop you from using bullish indicators to generate a SELL signal and bearish indicators to generate a BUY signal. 

In [ ]:
// Define a strategy with maximum of 15 days of history. It is important that these are enough days to calculate the various indicators.
// This example just shows how to use different indicators, they don't make much sense in this combination
val strategy = TaLibStrategy(15)

// We want to generate a BUY signal if we detect either one of the following two candlestick pattern: Morningstar or Three WhiteSoldiers
strategy.buy {
    cdlMorningStar(it) || cdl3WhiteSoldiers(it)
}

// We want to generate a SELL signal if we detect the candlestick pattern Two Crows and also observe that the 5 day weighted average is below the 9 day weighted average 
strategy.sell {
    cdl2Crows(it) && ema(it.close, 5) < ema(it.close, 9)
}


Now we can use this strategy just like any other strategy. In this case we use the S&P500 feed. 

Besides strategies, we can also use TALib indicators as metrics that are captured during a run. The following code creates a custom metric that calculates the Money Flow Index (aka mfi), one of the many indicators that comes with TALib. 

In [ ]:
val mfiMetric = TaLibMetric("mfi") {
    mfi(it)    
}

In [ ]:
val roboquant =  Roboquant(strategy, AccountMetric(), mfiMetric)
roboquant.run(feed)

Now lets see what the outcome is. Spoiler alert, this strategy won't make you rich ;) 

In [ ]:
val account = roboquant.broker.account
account.summary()

Let find out which trade generated most profit (realized PnL) and plot mfi for that asset

In [ ]:
// Lets find out the most profitable trade
val mostProfitableTrade = account.trades.maxByOrNull { it.pnl.value } !!
val asset = mostProfitableTrade.asset

// Lets plot the Money Flow Index for one of the assets
val symbol = asset.symbol.lowercase()
val metricData = roboquant.logger.getMetric("mfi.$symbol")
MetricChart(metricData)

## Bollinger Bands
This example is more realistic and shows how to implement a strategy based on Bollinger Bands. The underlying assumptions are:

1. If the current price is higher than the upper bound, it will continue to go up.
2. If the current price is lower than lower bound, it will continue to go down.

Of course you can easily reverse this logic from trend following to mean reversion if you prefer and compare the outcomes.

In [ ]:
// Define a strategy with maximum of bars of history. 
val barCount = 20
val strategy = TaLibStrategy(barCount)

// We want to BUY if current price is higher than the Bollinger Bands higher bound 
strategy.buy {
    val (h, m, l) = bbands(it, barCount)
    it.close.last() > h 
}

// We want to SELL if current price is lower than the Bollinger Bands lower bound 
strategy.sell {
    val (h, m, l) = bbands(it, barCount)
    it.close.last() < l   
}

In [ ]:
val roboquant =  Roboquant(strategy, AccountMetric())
roboquant.run(feed)

In [ ]:
val equity = roboquant.logger.getMetric("account.equity")
MetricChart(equity)

# Indicators

In [ ]:
// Indicator that returns multiple values
val asset = feed.assets.getBySymbol("GOOG")
val data = feed.apply(asset, TaLibIndicator.bbands())
MetricChart(data)

In [ ]:
// Compare indicators across different assets
val rsi = TaLibIndicator.rsi(20)
val data1 = feed.apply(feed.assets.getBySymbol("AAPL"), rsi)
val data2 = feed.apply(feed.assets.getBySymbol("TSLA"), rsi)
MetricChart(data1 + data2)

In [ ]:
// see where sma crossovers appear
val apple = feed.assets.getBySymbol("AAPL")
val data1 = feed.apply(apple, TaLibIndicator.sma(15))
val data2 = feed.apply(apple, TaLibIndicator.sma(50))
MetricChart(data1 + data2)